Exercise 1
---

**Chatbot dialog management**

Your job is to create a chatbot... As a first step, you will have to define the chatbot's framework.

In this exercise, you will have to write the necessary code to:
1. Read input from a user, as a chatbot would and display a simple response for an input. This should look like a conversation happening on a messenger.
2. Maintain dialogue state: which is the past "N" inputs from the user to the bot and the bot's responses
3. A few housekeeping commands for the bot:
    - Clear the context (/clearcontext)
    - Print out the context (/printcontext)
    - Configure the size of the context (/resizecontext N)
    - Quit the conversation (/quit)

In [4]:
from collections import deque

def process_statement_basic(x):
    if x[0] == "/":
        cont, response = process_command(x[1:])
    else:
        response = ["OK."]
        cont = True
    return cont, response


def process_command(x):
    parts = x.split()
    cmd = parts[0]
    args = parts[1:]
    
    
    global context
    if cmd == "quit":
        return False, ["Goodbye!"]
    elif cmd == "clearcontext":
        context.clear()
        return True, ["Cleared context."]
    elif cmd == "printcontext":
        response = context.get()
        response.append("Context length: %d" % (len(context)))
        return True, response
    elif cmd == "resizecontext":
        if len(args) == 1:
            
            try:
                context.resize(int(args[0]))
                return True, ["Resized context to %s" % (args[0])]
            except TypeError:
                return True, ["Context size should be int"]
        else:
            return True, ["resizecontext requires new size (int)"]
    else:
        return True, ["Invalid Command"]

class Context:
    
    def __init__(self, N):
        self.N = N
        self.context = None
        self.init()
    
    def __len__(self):
        return self.N
    
    def init(self):
        if self.context is None:
            self.context = deque(list(), self.N)
        else:
            self.context = deque(list(self.context), self.N)
        
    def add(self, x, response, cont, **kwargs):
        self.context.append({"x": x, "response": response, "cont": cont, 'tags': kwargs})
        
    def clear(self):
        self.context = None
        self.init()
    
    def resize(self, N):
        self.N = N
        self.init()

    def get(self):
        return list(self.context)
        
context = None

def start_chatbot(N = 10, name = 'Poincare'):
    global context
    context = Context(N)
    
    cont = True
    chat_cnt = 0
    
    while cont:
        x = input("You>> ")
        cont, response = process_statement_basic(x)
        for r in response:
            print("[", chat_cnt, "]", name, ">> ", r)
        context.add(x, response ,cont)
        chat_cnt += 1

In [82]:
if __name__=="__main__":
    # Start the bot...  
    start_chatbot()

You>> hi
[ 0 ] Poincare >>  OK.
You>> hello
[ 1 ] Poincare >>  OK.
You>> /resizecontext 2
[ 2 ] Poincare >>  Resized context to 2
You>> /printcontext
[ 3 ] Poincare >>  {'x': 'hello', 'tags': {}, 'cont': True, 'response': ['OK.']}
[ 3 ] Poincare >>  {'x': '/resizecontext 2', 'tags': {}, 'cont': True, 'response': ['Resized context to 2']}
[ 3 ] Poincare >>  Context length: 2
You>> /quit
[ 4 ] Poincare >>  Goodbye!


Exercise 2
---

**Adding basic intelligence to the chatbot**

Now that the dialogue is managed, let's add a few basic capabilities to the bot.

Poincare should now act as your calculator (with support for free form text). In other words, it should support the following operations:

1. Add, subtract, multiple, and divide two numbers and store it in the context (e.g. add 30 and 20; add the numbers 1000, 200, 3000; 30 + 50, 50 / 10, 5 times 20, and so on...)
2. Add the ability to manipulate/add-on to the result from the context (e.g. subtract 25 from the previous result)

**BONUS**

1. Build support for free-form dates (e.g. 2PM on Thursday, 5AM next Saturday, first sunday of next month, etc.); convert these strings to a Python "datetime" object

In [2]:
# This is left as an exercise...

Exercise 3
---

* Re-formulate the chatbot to ask a list of Yes/No questions that is procured from a file.
* State if the user's answer is correct or wrong.
* Make the source of input to the bot configurable. For now, it will come from the keyboard. Soon, we'll use our voice.

In [8]:
from collections import deque
from random import choice

class StatementProcessor:
    
    def __init__(self, N=10, statement_logic=lambda context, x: (True, ["OK"])):
            self.context = Context(N)
            self.statement_logic = statement_logic
    
    def process_statement(self, x):
        context = self.context
        if x[0] == "/":
            cont, response = self.process_command(x[1:])
        else:
            cont, response = self.statement_logic(context, x)
        return cont, response

    def process_command(self, x):
        context = self.context
        parts = x.split()
        cmd = parts[0]
        args = parts[1:]
        
        cont = False
        response = []
        
        if cmd == "quit":
            cont = False
            response = ["Goodbye!"]
        elif cmd == "clearcontext":
            context.clear()
            cont = True
            response = ["Cleared context."]
        elif cmd == "printcontext":
            response = context.get()
            response.append("Context length: %d" % (len(context)))
            cont = True
        elif cmd == "resizecontext":
            if len(args) == 1:
                try:
                    context.resize(int(args[0]))
                    cont = True
                    response = ["Resized context to %s" % (args[0])]
                except TypeError:
                    cont = True
                    response = ["Context size should be int"]
            else:
                cont = True
                response = ["resizecontext requires new size (int)"]
        else:
            cont = True
            response = ["Invalid Command"]
        context.add(x, response ,cont)
        return cont, response

    
def get_yes_no_processor(filename="binary_questions.txt"):
        qas = [v.split("#<>#") for v in filter(None, open(filename).read().split("\n"))]
        def statement_logic(context, x):
            prev_context = context.get()
            response = ["OK."]
            cont = True
            if len(prev_context)>0:
                prev_context = prev_context[-1]
            else:
                prev_context = {}
            #print(prev_context)
            if "tags" in prev_context and "question" in prev_context["tags"] and "qtype" in prev_context["tags"]["question"] and prev_context["tags"]["question"]["qtype"] == "binary":
                if prev_context["tags"]["question"]["expected_response"] == x:
                    response = ["Correct."]
                else:
                    response = ["Wrong.","Right answer is %s" %(prev_context["tags"]["question"]["expected_response"])]
                context.add(x, cont, response)
            else:
                qa_current = choice(qas)
                response = [qa_current[0]]
                response.append("Give the right answer.")
                context.add(x, cont, response, question={"expected_response": qa_current[1], "qtype": "binary"})
            return cont, response
        return statement_logic


def get_keyboard_source():
    def read_keyboard():
        #return raw_input("You>> ") # Python2
        return input("You>> ")
    return read_keyboard
    
class Bot:
    def __init__(self, name='Poincare', statement_processor=StatementProcessor(), input_source=get_keyboard_source()):
        self.name = name
        self.statement_processor = statement_processor
        self.input_source = input_source

    def start_bot(self):
        cont = True
        chat_cnt = 0

        while cont:
            x = self.input_source()
            cont, response = self.statement_processor.process_statement(x)
            for r in response:
                print("[", chat_cnt, "]", self.name, ">> ", r)
            chat_cnt += 1

In [9]:
if __name__=="__main__":
    bot = Bot(statement_processor=StatementProcessor(statement_logic=get_yes_no_processor()))
    bot.start_bot()

You>> Hi
[ 0 ] Poincare >>  Is it cold outside?
[ 0 ] Poincare >>  Give the right answer.
You>> no
[ 1 ] Poincare >>  Correct.
You>> /quit
[ 2 ] Poincare >>  Goodbye!
